In [6]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from poisson_binomial import poisson_binomial_pmf

In [22]:
p = np.array([0.3,0.6,0.2,0.1,0.9,0.5])
n = p.size
pmf = np.zeros(n+1)
poisson_binomial_pmf(p,n,pmf)
print(pmf,pmf.sum())

[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162] 1.0


In [ ]:
def poisson_binomial_likelihood(memory_pmf,memory_subpmf,memory_gradient):

In [23]:
pmf_lower = np.zeros((n,n)) # k,i
pmf_lower[0] = pmf[0]/(1.0-p)
for k in range(n-1):
    pmf_lower[k] = (pmf[k]-pmf_lower[k-1]*p)/(1.0-p)
pmf_lower[n-1] = 1.0-pmf_lower.sum(axis=0)

In [24]:
pmf_lower

array([[0.0144 , 0.0252 , 0.0126 , 0.0112 , 0.1008 , 0.02016],
       [0.1708 , 0.2719 , 0.1517 , 0.1364 , 0.3316 , 0.2276 ],
       [0.4058 , 0.4304 , 0.3812 , 0.3574 , 0.3686 , 0.443  ],
       [0.3238 , 0.2254 , 0.3402 , 0.3474 , 0.1666 , 0.2538 ],
       [0.0798 , 0.0444 , 0.1062 , 0.1314 , 0.0306 , 0.0522 ],
       [0.0054 , 0.0027 , 0.0081 , 0.0162 , 0.0018 , 0.00324]])

In [28]:
pmf_lower[:,2]

array([0.0126, 0.1517, 0.3812, 0.3402, 0.1062, 0.0081])

In [27]:
p_short = np.array([0.3,0.6,0.1,0.9,0.5])
n_short = p_short.size
pmf_short = np.zeros(n_short+1)
poisson_binomial_pmf(p_short,n_short,pmf_short)
print(pmf_short,pmf_short.sum())

[0.0126 0.1517 0.3812 0.3402 0.1062 0.0081] 0.9999999999999999
